In [ ]:
# generator function 

class sample_generator():
    def __init__(self,dataframe):
        
        self.dataframe = dataframe
        
    def next_batch():
        """ augment and return batch"""
        
        X_batch, ybatch =  batch_from_prefetch_samples(dataframe,self.batchsize)
        return augment_batch(X_batch) , y_batch
        
        
    def batch_from_prefetch_samples():
        dataframe = self.dataframe
        batchsize = self.batchsize
        y_batch = []
        X_batch = []
    
        serie = dataframe.sample(n = batchsize, replace = True )

        y_batch = serie[0].tolist()
        X_batch = serie[1].tolist()


        return  X_batch, y_batch

        
        
    def augment_batch(X_batch, add_shift = True, add_noise = True):
            """ make some noise, shift the data around """
            
            shift_max = self.shift_max
            noise_max = self.noise_max
            
            target_length = X_batch[0].shape[0]
    
            X_batch_out = []
    
            for X_elem in X_batch:
        
                if add_shift:
                    shift = random.randint(0,shift_max)
            #print(shift)
                else:
                    shift = 0
                deletion = []
                end = 0
                start = 1
        
            # cut randomly in the front and back of the columns
                for f in range(shift):
                    if random.uniform(0.1, 1.1) > 0.55:
                        deletion.append(  X_elem.shape[0] - (end+1) )
                
                        #X_elem = np.delete(X_elem,(X_elem.shape[0] - (end+1),0)

                        end +=1
                    else:
                        deletion.append(start)
                        #X_elem = np.delete(X_elem,(start),0)
        
                        start +=1
        
                X_elem = np.delete(X_elem,deletion,0)
                # randomly add the missing columns to the end or beginning of the dataset
        
                missing = target_length - X_elem.shape[0]
                if missing > 0 :
                    if random.uniform(0, 1) > 0.5:
                        # add to the front
                        X_elem = np.vstack((X_elem,np.random.rand(missing,13)))
                    
                    else:
                        # add to the back
                        X_elem = np.vstack((np.random.rand(missing,13), X_elem))
                    
                    if add_noise:
                        noise = np.random.normal(0,noise_max,(99*13)).reshape((99, 13))
                    #X_batch_out.append(np.add(noise,X_elem))
            
                else:
                    missing = X_elem.shape[0] -target_length 
                    
                    
                delete= []
                for element in range(missing):
                    delete.append(X_elem.shape[0] - (element) )
                    #print("element too long,removing one ")
                    X_elem = np.delete(X_elem,delete,0)

                if add_noise:
                    noise = np.random.normal(0,noise_max,(99*13)).reshape((99, 13))
                    
            try:
                X_batch_out.append(np.add(noise,X_elem))
            except:
                X_elem = np.delete(X_elem,X_elem.shape[0]-1,0)
                X_batch_out.append(np.add(noise,X_elem))
    
            return X_batch_out
        #
        #X_out = np.add(X_elem,noise)


In [ ]:
# write that thing into a class, for optimized optimization


class RNN_BN():
    def __init__(self, exampledata , learning_rate=0.0002, base_neurons = 40, momentum = 0.99, dropout = 0.4, batchsize = 100, outputs = 11
                rnn_layers = 3 , dense_layers = 2, n_epochs = 100, 
                 divisionfactor = 4 ,autorun = True, timecourse = False, 
                 savename = 'default_RNN', savethreshold = 0.9, n_filters = 20):
        
        # feed an example data, a list of ndarrays, generate tensors based on dimensions of first element
        # placeholders
        self.X = tf.placeholder(tf.float32,[None,exampledata[0][0],exampledata[0][1]])
        self.y = tf.placeholder(tf.int32,[None])
        self.learning_rate = learning_rate
        self.base_neurons = base_neurons
        self.momentum = momentum
        self.dropout = dropout
        self.batchsize = batchsize
        self.outputs = outputs
        self.rnn_layers = rnn_layers
        self.dense_layers = dense_layers
        self.timecourse = timecourse
        self.savename = savename
        self.n_epochs = n_epochs
        self.divisionfactor = dicisionfactor
        self.savethreshold = savethreshold
        self.n_filters = n_filters
        
        # autorun constructs 
        
        if autorun:
            construct()
        
        
    def construct(self):
        # clean the default graph
        tf.reset_default_graph()
        
        X = self.X
        y = self.y
        
        
        # training tensor
        training = tf.placeholder_with_default(False,shape=(),name="training")
        
        
        
        cells=[tf.contrib.rnn.GRUCell(num_units=n_neurons,activation=tf.nn.relu)]
        for c in range(self.rnn_layers):
            cells.append(tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.GRUCell(num_units=(n_neurons),
                                                                               activation=tf.nn.relu, 
                                                                               kernel_initializer = tf.contrib.layers.xavier_initializer()) 
                                                       ,input_keep_prob = (1-dropout) )  )
        
        #within scope assemble the multilayer cell
        with tf.name_scope("rnn"):
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=False)
            
            # dynamic RNN creates graphs faster than the static rnn 
            outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, self.X, dtype=tf.float32)
            

        with tf.name_scope("supplement_layer"):
            # dense layer  try states (last information of RNN  or outputs,  all information)
            
            if(timecourse):
                outputs_flat = tf.reshape(outputs, [-1, (n_neurons * n_inputs)])
            else:
                outputs_flat = states
            
            dense = tf.layers.dense(outputs_flat, (n_neurons - 5),
                                    kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                    name="supplementationDense1") # add a dense layer
            
    
    
    
               # batch normalization
            bn_norm_sup = tf.layers.batch_normalization(dense,training=training, momentum = momentum)
            bn_norm_act = tf.nn.elu(bn_norm_sup) # activate the batch norm
    
            drop1 = tf.layers.dropout(bn_norm_act, training = training, rate=0.5 )

            dense2 = tf.layers.dense(drop1, (n_neurons - 15), activation=tf.nn.elu,
                                     kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                     name="supplementationDense2") # add a dense layer
    
               # batch normalization
            bn_norm_sup2 = tf.layers.batch_normalization(dense2,training=training, momentum = momentum)
                             
            bn_norm_act2 = tf.nn.elu(bn_norm_sup2) # activate the batch norm
    
            #drop2 = tf.layers.dropout(bn_norm_act2, training = training, rate=0.5 )

            logits_before_bn = tf.layers.dense(bn_norm_act2, n_outputs, name="outputs_logits") #logits = tf.layers.dense(states, n_outputs)
            logits = tf.layers.batch_normalization(logits_before_bn,training=training, momentum = momentum)
    
            
            
        with tf.name_scope("loss"):
            # sparse softmax, so I don't have to get the ohot
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
            loss = tf.reduce_mean(xentropy)
    
    
        with tf.name_scope("train"):
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)

        with tf.name_scope("eval"):
            correct = tf.nn.in_top_k(logits, y, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
        with tf.name_scope("predict"):
            prediction = tf.argmax(logits,1)

        init = tf.global_variables_initializer()
            
        def run_model(self,fulldatasize = 1000000,generator,X_test,y_test):
            
            with tf.Session() as sess:
                init.run()
                
                for epoch in range(n_epochs):
                    for iteration in range( fulldatasize // (batch_size*40) ):
                        X_batch, y_batch = generator.next()
                        X_batch = augment_batch(X_batch, shift_max=12, noise_max=3.5)
                        
                        
                    
                    
                    acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
                    acc_mess = accuracy.eval(feed_dict={X: X_mess, y: y_mess})
                    if acc_mess > self.savethreshold :
                        save_path = saver.save(sess, "%_checkpoint%s.ckpt" %(self.savename, acc_mess) )
            
                    if verbose:
                        print(epoch, "Train accuracy:", acc_train, " Test accuracy ", acc_mess)  
                        
                acc_mess = accuracy.eval(feed_dict={X: X_mess, y: y_mess})
                save_path = saver.save(sess, "%_final_%s.ckpt" %(self.savename, acc_mess) )
            
            
        
    
     
            